# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [24]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [25]:
os.getcwd()
os.chdir('C:\\Users\\mahon\\Desktop\\Springboard\\Guided Capstone\\Step 2\\data')
os.getcwd()

df = pd.read_csv('step3_output.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,Region,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,2,2,Hilltop Ski Area,Alaska,Alaska,2090,294,0,0.0,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.0,152.0,30.0,0
1,4,4,Sunrise Park Resort,Arizona,Arizona,11100,1800,0,0.0,0,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.0,104.0,80.0,1
2,5,5,Yosemite Ski & Snowboard Area,Northern California,California,7800,600,0,0.0,0,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.0,107.0,NaN,1
3,8,8,Boreal Mountain Resort,Sierra Nevada,California,7700,500,0,0.0,0,...,380.0,200.0,150.000000,54.0,400.0,49.0,NaN,150.0,200.0,1
4,9,9,Dodge Ridge,Sierra Nevada,California,8200,1600,0,0.0,0,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.0,140.0,NaN,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [3]:
# dfo = pd.get_dummies(df.state,dummy_na=0)
# df = pd.concat((df.drop(['state','Region'], axis=1),dfo))

In [26]:
df.drop(['Unnamed: 0','Unnamed: 0.1','Region'],axis=1,inplace=True)

In [90]:
dfo = pd.get_dummies(df.state)

state_list = df.state.unique()

df = pd.concat((df.drop(['state'],axis=1), dfo))

df.head()

AttributeError: 'DataFrame' object has no attribute 'state'

In [28]:
df.fillna(0,inplace=True)

In [57]:
df = df.iloc[:172,:]

In [58]:
df.tail()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
167,Hogadon Basin,8000.0,640.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168,Sleeping Giant Ski Resort,7428.0,810.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,Snow King Resort,7808.0,1571.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170,Snowy Range Ski & Recreation Area,9663.0,990.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,White Pine Ski Area,9500.0,1100.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [75]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [88]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [78]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [79]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test

y_pred = model.predict(X_test)

In [80]:
print(y_pred)

[54.42941618 41.89816618 57.24191618 52.96066618 52.10910368 11.89816618
 55.79660368 90.46066618 71.42941618 41.71066618 70.46066618 82.71066618
 48.02316618 70.89816618 47.08566618 56.10910368 97.96066618 45.27316618
 48.46066618 51.02316618 64.14816618 69.83566618 24.24191618 48.39035368
 45.64816618 87.83566618 56.96066618 43.17941618 64.03097868 53.96066618
 41.55441618 65.83566618 39.64816618 85.27316618 62.29660368 31.83566618
 72.46066618 72.36691618 55.96066618 47.89816618 34.40597868 38.10910368
 59.58566618]


## Review Model Outcomes — Iterate over additional models as needed

In [81]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [82]:
from sklearn.metrics import explained_variance_score, mean_absolute_error

In [83]:
_ = explained_variance_score(y_test,y_pred)
print(_)

0.678538317244942


In [133]:
_ = mean_absolute_error(y_test,y_pred)
print(_)
_ = explained_variance_score(y_test,y_pred)
print(_)

8.500478693437895
0.678538317244942


In [85]:
print(model.score(X_train,y_train))

print(lm)

0.7224106797187673
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [86]:
print('the intercept is: ',lm.intercept_)

the intercept is:  50.96066618217054


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept tvo provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [91]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coef = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

coef.sort_values('Coefficient')


,Coefficient
Idaho,0.000000e+00
Iowa,0.000000e+00
Maine,0.000000e+00
Maryland,0.000000e+00
Massachusetts,0.000000e+00
Michigan,0.000000e+00
Minnesota,0.000000e+00
Missouri,0.000000e+00
Montana,0.000000e+00
Nevada,0.000000e+00


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [95]:

state_list = ['Name','AdultWeekend','Idaho','Iowa','Maine','Maryland','Massachusetts','Michigan','Minnesota','Missouri','Montana','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','Ohio','Oregon','Pennsylvania','Rhode Island','South Dakota','Tennessee','Utah','Vermont','Virginia','Washington','West Virginia','Indiana','Illinois','Wyoming','Connecticut','Colorado','California','Arizona','Alaska','Wisconsin']
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X1 = df.drop(state_list, axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y1 = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X1)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled1=scaler.transform(X1) 

In [96]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y1 = y1.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X1_train, X1_test, y1_train, y1_test = train_test_split(X_scaled1, y1, test_size=0.25, random_state=1)

In [118]:
lm1 = linear_model.LinearRegression()
model1 = lm1.fit(X1_train,y1_train)

In [119]:
y_pred1 = model1.predict(X1_test)

In [120]:
print(y_pred1)

[ 54.53122069  40.54236834  56.98126949  52.23012062  51.34882969
  10.84354676  55.73656087  90.72576957  70.5545159   40.96420976
  71.3589488   82.83361282  47.42818741  70.53182749  47.03540734
  56.33374331 100.22164967  45.95786771  48.57966493  50.71569064
  64.72481502  70.73717289  23.19656384  49.00810539  46.26555114
  86.78435398  56.42458617  43.05973152  61.89456069  53.21802569
  41.34804139  65.68909855  39.12572933  85.91787955  61.09238168
  33.29423578  70.60398222  73.89019632  56.3549048   47.17793856
  34.29675825  37.47965287  59.11766318]


In [121]:
print(model1.score(X1_train,y1_train))

0.7237991556701907


In [122]:
print('the intercept is: ',model1.intercept_)

the intercept is:  51.02033105014927


In [123]:
_ = mean_absolute_error(y1_test,y_pred1)
print(_)
_ = explained_variance_score(y1_test,y_pred1)
print(_)

8.826440243168285
0.6528478449294142


In [124]:
coef = pd.DataFrame(abs(model1.coef_), X1.columns, columns=['Coefficient'])

coef.sort_values('Coefficient')

,Coefficient
trams,0.000000e+00
fastEight,4.440892e-16
fastSixes,3.552714e-15
TerrainParks,3.907988e-02
projectedDaysOpen,2.046178e-01
total_chairs,2.272021e-01
triple,2.364112e-01
quad,4.042198e-01
surface,5.377002e-01
Snow Making_ac,6.158586e-01


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [125]:

state_list = ['Name','AdultWeekend','AdultWeekday','summit_elev','AdultWeekday','Idaho','Iowa','Maine','Maryland','Massachusetts','Michigan','Minnesota','Missouri','Montana','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','Ohio','Oregon','Pennsylvania','Rhode Island','South Dakota','Tennessee','Utah','Vermont','Virginia','Washington','West Virginia','Indiana','Illinois','Wyoming','Connecticut','Colorado','California','Arizona','Alaska','Wisconsin']
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X2 = df.drop(state_list, axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y2 = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X2)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled2=scaler.transform(X2) 

In [126]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y2 = y2.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X2_train, X2_test, y2_train, y2_test = train_test_split(X_scaled2, y2, test_size=0.25, random_state=1)

In [127]:
lm2 = linear_model.LinearRegression()
model2 = lm2.fit(X2_train,y2_train)

In [128]:
y_pred2 = model2.predict(X2_test)

In [129]:
print(model2.score(X2_train,y2_train))

0.4130681834382397


In [130]:
print('the intercept is: ',model2.intercept_)

the intercept is:  50.18447078887821


In [131]:
_ = mean_absolute_error(y2_test,y_pred2)
print(_)
_ = explained_variance_score(y2_test,y_pred2)
print(_)

13.725493818215545
0.004474782774550712


In [132]:
coef = pd.DataFrame(abs(model2.coef_), X2.columns, columns=['Coefficient'])

coef.sort_values('Coefficient')

,Coefficient
trams,4.440892e-16
fastEight,1.776357e-15
fastSixes,3.996803e-15
surface,2.555432e-01
projectedDaysOpen,2.979087e-01
TerrainParks,5.410230e-01
yearsOpen,9.466841e-01
vertical_drop,1.259079e+00
triple,1.422146e+00
clusters,1.483860e+00


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.67 | 8.5 |-|
| Model 2. | 0.65 | 8.82 |'state'|
| Model 3. | 0.004 | 13.72 |'state','summit_elev','base_elev'|

Model Selection:

In [ ]:
I think this question really has two answers. Model 3 shows which uncontrollable features contribute to ticket prices - i.e. what management can do with the mountian to generate more revenue. But in terms of understanding what is a reasonable range for their ticket prices, the first model is the most robust in terms of predictive power. 